In [1]:
#Code Snippet 25 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkDateTime").getOrCreate()
df = spark.read.csv("items_bought.csv",header=True,inferSchema=True)
df.show(3)
print("Schema with date as string datatype")
df.printSchema()
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date #importing necessary functions to convert date string to date type
updated_df = df.withColumn('formatted_date',to_date(unix_timestamp(df['date'],'dd-MM-yyyy').cast('timestamp')))
print("Schema with date column string datatype converted to date datatype")
updated_df.printSchema()
print("Data after dropping the date column which was of string type")
updated_df=updated_df.drop("date") #dropping the date column with string type
updated_df.show(2)
from pyspark.sql.functions import weekofyear, dayofmonth,month,year,date_format #extracting data from dates
print("Data Extraction from dates")
final_df = updated_df.select(updated_df["item_name"], weekofyear(updated_df["formatted_date"]).alias("week_number"),          dayofmonth(updated_df["formatted_date"]).alias("day_number"),                  month(updated_df["formatted_date"]).alias("month"), year(updated_df["formatted_date"]).alias("year"))
final_df.show(3)
date_string_value = updated_df.select(df["item_name"],date_format(updated_df["formatted_date"],'MM/dd/yyyy')) #converting date type to a different date format string
date_string_value.show(2)
print("Usecase - Total amount of items purchased in that particular year")
final_format=final_df.groupBy("year").sum().select(["year","sum(year)"])
final_format.withColumnRenamed("sum(year)","Total Expenditure").show()

+----------+---------+----------+--------+----------+------------+
|      date|item_name|item_price|quantity|tax_amount|total_amount|
+----------+---------+----------+--------+----------+------------+
|11-10-2018|     Beer|     110.5|       2|     53.04|      163.54|
|14-02-2018|   Whisky|    1250.0|       1|     300.0|      1550.0|
|23-03-2020|   Whisky|    1300.5|       2|    624.24|     1924.74|
+----------+---------+----------+--------+----------+------------+
only showing top 3 rows

Schema with date as string datatype
root
 |-- date: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- tax_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)

Schema with date column string datatype converted to date datatype
root
 |-- date: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_price: double (nullable = true)
 |-- quantity: integer (nul